In [3]:
import json
import pickle
import logging
import argparse
import numpy as np
import pandas as pd
# import vllm

import string
import re
import collections
from sklearn.metrics import f1_score
# import Levenshtein
import os
from tqdm.notebook import tqdm

import datetime
from functools import reduce
# from huggingface_hub import login
# from dataclasses import dataclass
import pickle
import sys
from collections import defaultdict


In [4]:
def load_jsonl(path):
    with open(path, 'r') as f:
        a = f.readlines()
        g = [json.loads(i) for i in a]
    return g
eval_output_path = "./downstream_compare_outputs"
valfile = load_jsonl("./downstream_compare_outputs/val_downstream.jsonl")

In [5]:
!python -V

Python 3.12.2


In [6]:
valfile[0]

{'system': 'You are a linguist and a material scientist. You need to identify the named entity for each of the keywords given after WORDS in the input. Answer to the question should be from one of the provided options. Do not output anything else other than the answer. You should output the word entity pair separated by ":" in each line. Your options are: b-mat, i-mat, b-spl, i-spl, b-dsc, i-dsc, b-pro, i-pro, b-apl, i-apl, b-smt, i-smt, b-cmt, i-cmt. Answer for each word must be in a new line.',
 'question': 'WORDS: Ru\nSENTENCE: Ru ions appear to be in the pentavalent state .',
 'answer': 'Ru : b-mat',
 'task': 'downstream',
 'dataset': 'matscholar'}

In [7]:
# import os

# outputs = {}

# for fname in tqdm(os.listdir(eval_output_path)):
#     fpath = os.path.join(eval_output_path, fname)
#     model_name = fname.split('_downstream_eval.pkl')[0]
#     with open(fpath, 'rb') as f:
#         outputs[model_name] = pickle.load(f)
#     print(f"{model_name:<20}", "Done")
    

In [8]:
outputs = json.load(open('./downstream_compare_outputs/_downstream_eval.json','r'))

In [9]:
outputs['llamat3-chat']['ner'].keys()

dict_keys(['matscholar', 'sofc_token', 'sc_comics'])

In [10]:
keylist = [list(val.keys()) for val in valfile]
valfile[19]

{'system': 'You are a linguist and a material scientist. You need to identify the named entity for each of the keywords given after WORDS in the input. Answer to the question should be from one of the provided options. Do not output anything else other than the answer. You should output the word entity pair separated by ":" in each line. Your options are: b-material, i-material, b-device, i-device, b-experiment, i-experiment, b-value, i-value. Answer for each word must be in a new line.',
 'question': 'WORDS: demonstrate, BCFZY-ZnO, SOFC, 400, –, 500, °, C, .\nSENTENCE: However , our findings reveal it is feasible to demonstrate BCFZY-ZnO electrolyte in SOFC with excellent ionic conductivity , high OCVs and power densities at low temperature range of 400 – 500 ° C .',
 'answer': 'demonstrate : b-experiment\nBCFZY-ZnO : b-material\nSOFC : b-device\n400 : b-value\n– : i-value\n500 : i-value\n° : i-value\nC : i-value\n. : i-value',
 'task': 'downstream',
 'dataset': 'sofc_token'}

In [11]:
np.unique(keylist)

array(['answer', 'dataset', 'question', 'system', 'task'], dtype='<U8')

In [12]:
keylist[0]

['system', 'question', 'answer', 'task', 'dataset']

In [13]:
outputs['llamat3-chat']['ner']['sofc_token']

[{'sample_idx': 19,
  'gold_answer': 'demonstrate : b-experiment\nBCFZY-ZnO : b-material\nSOFC : b-device\n400 : b-value\n– : i-value\n500 : i-value\n° : i-value\nC : i-value\n. : i-value',
  'prediction': 'demonstrate : b-experiment\nBCFZY-ZnO : b-material\nSOFC : b-device\n400 : b-value\n– : i-value\n500 : i-value\n° : i-value\nC : i-value\n. : i-value',
  'processed_gold': [['demonstrate', 'b-experiment'],
   ['bcfzy-zno', 'b-material'],
   ['sofc', 'b-device'],
   ['400', 'b-value'],
   ['–', 'i-value'],
   ['500', 'i-value'],
   ['°', 'i-value'],
   ['c', 'i-value'],
   ['.', 'i-value']],
  'processed_prediction': [['demonstrate', 'b-experiment'],
   ['bcfzy-zno', 'b-material'],
   ['sofc', 'b-device'],
   ['400', 'b-value'],
   ['–', 'i-value'],
   ['500', 'i-value'],
   ['°', 'i-value'],
   ['c', 'i-value'],
   ['.', 'i-value']],
  'system_prompt': 'You are a linguist and a material scientist. You need to identify the named entity for each of the keywords given after WORDS in th

In [14]:
our = 'llamat3-chat'
def get_correct_indices(analysis_dict, task, dataset):
    correct = []
    for entry in analysis_dict[task][dataset]:
        if entry["processed_gold"] == entry["processed_prediction"]:
            correct.append(entry["sample_idx"])
    return correct

In [15]:
base_llamas = ['LLaMA-3-chat', 'LLaMA-2',  'LLaMA-2-chat', 'LLaMA-3']
all_others = ['Claude-3-Opus', 'GPT-4o', 'Gemini-1.5-Flash-8b', 'Claude-3-Haiku', 'LLaMA-3-chat', 'Gemini-1.5-flash', 'GPT-4',  'LLaMA-2',  'Gemini-1.5-pro', 'LLaMA-2-chat', 'Claude-3.5-Sonnet', 'LLaMA-3']

In [16]:
def compare_across_all(analysis_dicts, our_name, other_name):
    results = {}
    for task in analysis_dicts[our_name].keys():
        results[task] = {}
        for dataset in analysis_dicts[our_name][task].keys():
            ours_better = compare_models(
                analysis_dicts[our_name], 
                analysis_dicts[other_name], 
                task, dataset
            )
            results[task][dataset] = ours_better
    return results

In [17]:
def find_unique_wins(analysis_dicts, our_name, other_names, min_wrong=None):
    results = []

    for task in analysis_dicts[our_name]:
        for dataset in analysis_dicts[our_name][task]:
            for entry in analysis_dicts[our_name][task][dataset]:
                idx = entry["sample_idx"]
                gold = entry["processed_gold"]
                ours_pred = entry["processed_prediction"]

                # skip if our model is wrong
                if ours_pred != gold:
                    continue

                wrong_count = 0
                compared = 0
                for other in other_names:
                    # check if this other model has that idx
                    other_entries = [
                        e for e in analysis_dicts[other][task][dataset]
                        if e["sample_idx"] == idx
                    ]
                    if not other_entries:  # skip if missing
                        continue

                    compared += 1
                    other_entry = other_entries[0]
                    if other_entry["processed_prediction"] != gold:
                        wrong_count += 1

                if compared == 0:
                    continue  # nobody to compare against

                if min_wrong is None:
                    if wrong_count == compared:  # all wrong
                        results.append((task, dataset, idx))
                else:
                    if wrong_count >= min_wrong:
                        results.append((task, dataset, idx))

    return results


In [18]:
results = find_unique_wins(outputs, 'llamat3-chat', all_others)

In [19]:
len(results)

1733

In [20]:
def find_unique_wins_with_stats(analysis_dicts, our_name, other_names, min_wrong=None, return_full=False):
    """
    Returns cases where our model is correct and others are wrong,
    and also generates a dict structured like original analysis_dict,
    along with statistics.

    Parameters
    ----------
    analysis_dicts : dict
        Dictionary like your `analysis_dict` containing all models' outputs.
    our_name : str
        The key of our model in the dictionary.
    other_names : list of str
        Keys of other models to compare against.
    min_wrong : int or None
        Minimum number of other models that must be wrong. If None, all must be wrong.
    return_full : bool
        If True, returns full data including predictions and gold.

    Returns
    -------
    wins_list : list
        List of unique wins (see previous function).
    wins_dict : dict
        Same structure as analysis_dict but only containing unique wins.
    stats : dict
        {task: {dataset: (unique_count, total_count, fraction)}} 
    """
    wins_list = []
    wins_dict = defaultdict(lambda: defaultdict(list))
    stats = defaultdict(lambda: defaultdict(lambda: (0, 0, 0.0)))

    for task in analysis_dicts[our_name]:
        for dataset in analysis_dicts[our_name][task]:
            total = len(analysis_dicts[our_name][task][dataset])
            unique_count = 0

            for entry in analysis_dicts[our_name][task][dataset]:
                idx = entry["sample_idx"]
                gold = entry["processed_gold"]
                ours_pred = entry["processed_prediction"]

                if ours_pred != gold:
                    continue

                wrong_count = 0
                compared = 0
                other_data = {}

                for other in other_names:
                    other_entries = [
                        e for e in analysis_dicts[other][task][dataset]
                        if e["sample_idx"] == idx
                    ]
                    if not other_entries:
                        continue

                    compared += 1
                    other_entry = other_entries[0]
                    other_pred = other_entry["processed_prediction"]

                    other_data[other] = other_pred

                    if other_pred != gold:
                        wrong_count += 1

                if compared == 0:
                    continue  # nobody to compare against

                condition = (
                    (min_wrong is None and wrong_count == compared) or
                    (min_wrong is not None and wrong_count >= min_wrong)
                )

                if condition:
                    unique_count += 1
                    if return_full:
                        wins_list.append({
                            "task": task,
                            "dataset": dataset,
                            "sample_idx": idx,
                            "gold": gold,
                            "ours": ours_pred,
                            "others": other_data
                        })
                    else:
                        wins_list.append((task, dataset, idx))

                    # add to wins_dict in the same structure as analysis_dict
                    wins_dict[task][dataset].append(entry)

            fraction = unique_count / total if total > 0 else 0
            stats[task][dataset] = (unique_count, total, fraction)

    # Print stats nicely
    print("=== Unique Wins Statistics ===")
    for task in stats:
        print(f"\nTask: {task}")
        for dataset in stats[task]:
            unique_count, total, fraction = stats[task][dataset]
            print(f"  Dataset: {dataset}, Unique Wins: {unique_count}/{total} ({fraction:.2%})")

    return wins_list, wins_dict, stats


In [21]:
results = find_unique_wins_with_stats(outputs, 'llamat3-chat', all_others)

=== Unique Wins Statistics ===

Task: ner
  Dataset: matscholar, Unique Wins: 582/1070 (54.39%)
  Dataset: sofc_token, Unique Wins: 25/175 (14.29%)
  Dataset: sc_comics, Unique Wins: 400/945 (42.33%)

Task: pc
  Dataset: glass_non_glass, Unique Wins: 17/300 (5.67%)

Task: sf
  Dataset: sofc_token, Unique Wins: 41/175 (23.43%)

Task: ee
  Dataset: sc_comics, Unique Wins: 1/315 (0.32%)

Task: re
  Dataset: structured_re, Unique Wins: 0/1529 (0.00%)
  Dataset: sc_comics, Unique Wins: 17/418 (4.07%)

Task: sar
  Dataset: synthesis_actions, Unique Wins: 123/566 (21.73%)

Task: sc
  Dataset: sofc_sent, Unique Wins: 437/1855 (23.56%)

Task: qna
  Dataset: squad, Unique Wins: 81/1081 (7.49%)

Task: mcq
  Dataset: story_cloze, Unique Wins: 0/324 (0.00%)
  Dataset: hellaswag, Unique Wins: 7/976 (0.72%)
  Dataset: boolqa, Unique Wins: 2/500 (0.40%)
